# Games

In [244]:
import concurrent.futures
import os
import requests

In [245]:
USERNAMES = ["magnuscarlsen", "thedarkknighttrilogy", "hikaru", "wonderfultime", "danielnaroditsky"]
USERNAMES.sort()

In [246]:
def get_archives(username : str):
    archives_url = f'https://api.chess.com/pub/player/{username}/games/archives'
    response = requests.get(archives_url)
    archives_data = response.json()
    archives = archives_data.get("archives")
    archives.reverse()
    return archives

In [247]:
def save_pgn(options : dict):
    username : str = options.get("username", "")
    archive : str = options.get("archive", "")
    paths : list[str] = archive.split("/")
    year_month : list[str] = paths[-2:]
    [year, month] = year_month
    pgn_url : str= f"{archive}/pgn"
    response : requests.Response = requests.get(pgn_url)
    pgn_text : str = response.text.strip()
    pgn_path : str = f"./{username}/games"
    pgn_exists : bool = os.path.exists(pgn_path)
    if not pgn_exists:
        os.makedirs(pgn_path)
    if pgn_text == "":
        return
    pgn_file = open(f"{pgn_path}/{year}-{month}.pgn", "w")
    pgn_file.write(pgn_text)
    pgn_file.close()
    print(year, month, 'done')

In [248]:
def save_all_pgn(username_archives : list[dict]):
    number_of_workers = len(username_archives)
    with concurrent.futures.ThreadPoolExecutor(max_workers=number_of_workers) as executor:
        executor.map(save_pgn, username_archives)

In [249]:
for username in USERNAMES:
    print("username", username)
    archives = get_archives(username)
    username_archives = []
    for archive in archives:
        username_archives.append({
            "username": username,
            "archive": archive
        })
    save_all_pgn(username_archives=username_archives)

username danielnaroditsky
2015 05 done
2014 09 done
2013 09 done
2015 09 done
2014 05 done
2013 11 done
2014 04 done
2013 12 done
2014 03 done
2012 10 done
20152015 08 done
 03 done
2013 08 done
2015 06 done
2014 01 done
2016 01 done
2011 10 done
2014 12 done
2015 12 done
2010 06 done
2012 01 done
2011 08 done
2013 10 done
2015 02 done
2015 11 done
2011 02 done
2011 01 done
2016 02 done
2017 11 done
2014 02 done
2017 06 done
2018 01 done
2018 03 done
2017 03 done
2017 02 done
2023 07 done
2017 09 done
2016 03 done
2017 07 done
2018 02 done
2017 04 done
2016 07 done
2017 10 done
2017 01 done
2017 12 done
2019 06 done
2016 10 done
2016 05 done
2018 04 done
2016 09 done
2016 04 done
2017 05 done
2022 06 done
2021 11 done
2021 08 done
2016 12 done
2016 08 done
2018 06 done
2022 04 done
2021 07 done
2019 01 done
2022 11 done
2023 02 done
2018 10 done
2021 10 done
2020 01 done
2022 08 done
2022 05 done
2023 03 done
2022 12 done
2022 07 done
2020 09 done
2018 05 done
2020 03 done
2021 12 done